In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np


In [3]:
normal = np.load('normal_data_scaled_freq_domain.npy')
horizontal_misalignment_data = np.load('horizontal_misalignment_data_data_scaled_freq_domain.npy',)
imbalance_data = np.load('imbalance_data_scaled_freq_domain.npy')
vertical_misalignment = np.load('vertical_misalignment_data_scaled_freq_domain.npy')

In [5]:
len(horizontal_misalignment_data)

197

In [6]:
#labels
y_normal = np.full(len(normal), "normal")
y_horizontal_misalignment = np.full(len(horizontal_misalignment_data),'horizontal_misalignment')
y_imbalance = np.full(len(imbalance_data),'imbalance')
y_vertical_misalignment = np.full(len(vertical_misalignment),'vertical_misalignment')

In [15]:
len(normal)

49

we used 50 instances of the faulty data cuz len(normal data == 49)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load your data
normal = np.load('normal_data_scaled_freq_domain.npy')
horizontal_misalignment_data = np.load('horizontal_misalignment_data_data_scaled_freq_domain.npy')
imbalance_data = np.load('imbalance_data_scaled_freq_domain.npy')
vertical_misalignment_data = np.load('vertical_misalignment_data_scaled_freq_domain.npy')

# Concatenate the datasets
data = np.concatenate([normal, horizontal_misalignment_data[:20], imbalance_data[:20], vertical_misalignment_data[:20]], axis=0)

# Create labels
y_normal = np.full(49, "normal")
y_horizontal_misalignment = np.full(20, 'horizontal_misalignment')
y_imbalance = np.full(20, 'imbalance')
y_vertical_misalignment = np.full(20, 'vertical_misalignment')

labels = np.concatenate([y_normal, y_horizontal_misalignment, y_imbalance, y_vertical_misalignment])

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Convert data and labels to PyTorch tensors
data_tensor = torch.FloatTensor(data)
labels_tensor = torch.LongTensor(encoded_labels)

# Reshape data for LSTM (assuming it's a time series)
data_tensor = data_tensor.view(data_tensor.shape[0], 1, -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2, random_state=42, stratify=labels_tensor)

# Define LSTM model
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Instantiate the model
input_size = X_train.size(2)
hidden_size = 50
num_classes = len(np.unique(encoded_labels))
model = SimpleLSTM(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test.numpy(), predicted.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch [1/1000], Loss: 1.3748
Epoch [2/1000], Loss: 1.3411
Epoch [3/1000], Loss: 1.3221
Epoch [4/1000], Loss: 1.3073
Epoch [5/1000], Loss: 1.2966
Epoch [6/1000], Loss: 1.2850
Epoch [7/1000], Loss: 1.2731
Epoch [8/1000], Loss: 1.2623
Epoch [9/1000], Loss: 1.2544
Epoch [10/1000], Loss: 1.2477
Epoch [11/1000], Loss: 1.2403
Epoch [12/1000], Loss: 1.2322
Epoch [13/1000], Loss: 1.2250
Epoch [14/1000], Loss: 1.2180
Epoch [15/1000], Loss: 1.2111
Epoch [16/1000], Loss: 1.2040
Epoch [17/1000], Loss: 1.1978
Epoch [18/1000], Loss: 1.1926
Epoch [19/1000], Loss: 1.1860
Epoch [20/1000], Loss: 1.1811
Epoch [21/1000], Loss: 1.1741
Epoch [22/1000], Loss: 1.1681
Epoch [23/1000], Loss: 1.1616
Epoch [24/1000], Loss: 1.1559
Epoch [25/1000], Loss: 1.1509
Epoch [26/1000], Loss: 1.1454
Epoch [27/1000], Loss: 1.1387
Epoch [28/1000], Loss: 1.1316
Epoch [29/1000], Loss: 1.1242
Epoch [30/1000], Loss: 1.1175
Epoch [31/1000], Loss: 1.1115
Epoch [32/1000], Loss: 1.1063
Epoch [33/1000], Loss: 1.1012
Epoch [34/1000], Lo

In [13]:
# Define a more complex LSTM model
class AdvancedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.5):
        super(AdvancedLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(self.relu(out))
        return out

# Instantiate the model
input_size = X_train.size(2)
hidden_size = 128
num_layers = 2
num_classes = len(np.unique(encoded_labels))
model = AdvancedLSTM(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test.numpy(), predicted.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch [1/200], Loss: 1.3948
Epoch [2/200], Loss: 1.3883
Epoch [3/200], Loss: 1.3750
Epoch [4/200], Loss: 1.3692
Epoch [5/200], Loss: 1.3575
Epoch [6/200], Loss: 1.3507
Epoch [7/200], Loss: 1.3445
Epoch [8/200], Loss: 1.3277
Epoch [9/200], Loss: 1.3164
Epoch [10/200], Loss: 1.3126
Epoch [11/200], Loss: 1.3071
Epoch [12/200], Loss: 1.2937
Epoch [13/200], Loss: 1.2880
Epoch [14/200], Loss: 1.2779
Epoch [15/200], Loss: 1.2634
Epoch [16/200], Loss: 1.2621
Epoch [17/200], Loss: 1.2559
Epoch [18/200], Loss: 1.2476
Epoch [19/200], Loss: 1.2384
Epoch [20/200], Loss: 1.2352
Epoch [21/200], Loss: 1.2265
Epoch [22/200], Loss: 1.2172
Epoch [23/200], Loss: 1.2213
Epoch [24/200], Loss: 1.1985
Epoch [25/200], Loss: 1.2106
Epoch [26/200], Loss: 1.1939
Epoch [27/200], Loss: 1.1913
Epoch [28/200], Loss: 1.1863
Epoch [29/200], Loss: 1.1964
Epoch [30/200], Loss: 1.1853
Epoch [31/200], Loss: 1.1768
Epoch [32/200], Loss: 1.1693
Epoch [33/200], Loss: 1.1594
Epoch [34/200], Loss: 1.1378
Epoch [35/200], Loss: 1